# Metric Viewer
In this notebook we will view the metrics of all predictions

## Imports

In [ ]:
import evalseg
from evalseg.metrics import MME,MultiClassMetric
import numpy as np
import pandas as pd
from ipywidgets import interact, interact_manual, IntSlider
import os

## Variables

In [ ]:
root_data = '../datasets'

## Run Metric

In [ ]:
@interact
def select_dataset(dataset_name=sorted([d for d in os.listdir(root_data) if os.path.isdir(f'{root_data}/{d}')])):
    global dataset
    evalseg.reload()
    dataset = evalseg.io.NibDataset(f'{root_data}/{dataset_name}')

    @interact
    def select_case(case=sorted(dataset.get_available_ids())):
        dataset.load_all_of_case(case)
        gto, gto_voxelsize = dataset.get_groundtruth(case)
        cto, cto_voxelsize = dataset.get_CT(case)
        gtmax = (gto.sum(axis=2).sum(axis=1).argmax(), gto.sum(axis=2).sum(axis=0).argmax(), gto.sum(axis=1).sum(axis=0).argmax())

        @interact
        def axselect(ax=['z', 'x', 'y', 'all']):
            axi = {'all': -1, 'x': 0, 'y': 1, 'z': 2}[ax]
            @interact
            def onlyMaxFrame(frame=IntSlider(-1,min=-1,max=-1) if ax=='all' else IntSlider(gtmax[axi], min=0, max=gto.shape[axi]-1)):
                global data, gt, voxelsize
                gt, voxelsize = evalseg.geometry.slice(gto, gto_voxelsize, axi, [frame])
                ct, _ = evalseg.geometry.slice(cto, None, axi, [frame])
                data = {p: evalseg.geometry.slice(dataset.get_prediction(p, case)[0], None, axi, [frame])[0] for p in dataset.get_prediction_methods()}
                if ax!='all':
                    ctlbl = f"{dataset_name.split('_')[1]} {case} {ax}={frame}"
                    evalseg.ui.multi_plot_2d(ct, gt, data, spacing=voxelsize, ctlbl= ctlbl, z_titles= [frame])
                else:
                    pass #TODO
                mme = evalseg.metrics.MultiClassMetric(evalseg.metrics.MME,dataset.num_labels)
                mme.set_reference(gt, voxelsize)
                res = mme.evaluate_multi(data)
                evalseg.ui.plot_metric_multi(res, name=dataset_name)
